In [54]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import pandas as pd
from random import randint
from time import sleep

# DATE VARIABLES MUST BE YYYY-MM-DD
STARTDATE = '2021-01-01'
ENDDATE = '2021-07-01' # up to, not including
CSV_TITLE = 'differences_data_2021_01_06.csv'

# Put points into buckets of: 
BUCKET_BY = 5

#Date Stuff
DAYS_PER_MONTH = {'01': 31, '02': 28, '03': 31, '04': 30, '05': 31, '06': 30, '07': 31, '08': 31, '09': 30, 
                  '10': 31, '11': 30, '12': 31}
DAYS_PER_MONTH_leap = {'01': 31, '02': 29, '03': 31, '04': 30, '05': 31, '06': 30, '07': 31, '08': 31, '09': 30,
                       '10': 31,'11': 30, '12': 31}
LEAP_YEAR = ['2020']

In [55]:
teams = ['Atlanta', 'Boston', 'Brooklyn', 'Charlotte', 'Chicago', 'Cleveland', 'Dallas', 'Denver', 'Detroit', 
         'Golden State', 'Houston', 'Indiana', 'LA Clippers', 'LA Lakers', 'Memphis', 'Miami', 'Milwaukee', 
         'Minnesota', 'New Orleans', 'New York', 'Oklahoma City', 'Orlando', 'Philadelphia', 'Phoenix', 'Portland', 
         'Sacramento', 'San Antonio', 'Toronto', 'Utah', 'Washington']
ids = [x for x in range(1,31)]
TEAM_IDS = dict(zip(teams, ids))

In [56]:
def get_date_array():
    temp_date = STARTDATE
    dates = []
    while temp_date != ENDDATE:
        dates.append(temp_date)
        year, month, day = temp_date.split('-')
        # account for leap_years
        if year in LEAP_YEAR:
            days = DAYS_PER_MONTH_leap
        else:
            days = DAYS_PER_MONTH
        # move to next date
        if days[month] > int(day):
            temp_date = '%s-%s-%02d' % (year, month, int(day)+1)
        else:
            if int(month) >= 12: #==
                temp_date = '%s-01-01' % str(int(year)+1)
            else:
                temp_date = '%s-%02d-01' % (year, int(month)+1)
    return dates

In [57]:
def reformat_dates(date_array):
    ret = []
    for date in date_array:
        parts = date.split('-')
        ret.append('month=%s&day=%s&year=%s' % (parts[1], parts[2], parts[0]))
    return ret     

In [58]:
def get_finals():
    columns = ['Team_x', 'Points_x', 'Team_y', 'Points_y', 'Date', 'Home', 'Winner']
    dates = get_date_array()
    dates_for_url = reformat_dates(dates)
    ret = []
    url_finals = 'https://www.basketball-reference.com/boxscores/?'
    for i in range(len(dates_for_url)):
        print('Scraping for: %s' % dates[i])
        html = urllib.urlopen(url_finals+dates_for_url[i])
        soup = BeautifulSoup(html.read())
        html.close()
        scores = soup.find_all('table', class_='teams')
        for score in scores:
            table = score.get_text()
            team1 = table.split('\n')[3:5]
            team2 = table.split('\n')[10:12]
            home_team = team2[0]
            if int(team1[1]) > int(team2[1]):
                winner = team1[0]
            else:
                winner = team2[0]
            
            temp = team1+team2
            temp2 = team2+team1
            
            temp.append(dates[i])
            temp.append(home_team)
            temp.append(winner)
            ret.append(temp)
            
            temp2.append(dates[i])
            temp2.append(home_team)
            temp2.append(winner)
            ret.append(temp2)
            
        sleep(randint(1,3))
    #print(ret)
    final = pd.DataFrame(ret, columns=columns)
    return final

In [59]:
finals = get_finals()
finals

Scraping for: 2021-01-01
Scraping for: 2021-01-02
Scraping for: 2021-01-03
Scraping for: 2021-01-04
Scraping for: 2021-01-05
Scraping for: 2021-01-06
Scraping for: 2021-01-07
Scraping for: 2021-01-08
Scraping for: 2021-01-09
Scraping for: 2021-01-10
Scraping for: 2021-01-11
Scraping for: 2021-01-12
Scraping for: 2021-01-13
Scraping for: 2021-01-14
Scraping for: 2021-01-15
Scraping for: 2021-01-16
Scraping for: 2021-01-17
Scraping for: 2021-01-18
Scraping for: 2021-01-19
Scraping for: 2021-01-20
Scraping for: 2021-01-21
Scraping for: 2021-01-22
Scraping for: 2021-01-23
Scraping for: 2021-01-24
Scraping for: 2021-01-25
Scraping for: 2021-01-26
Scraping for: 2021-01-27
Scraping for: 2021-01-28
Scraping for: 2021-01-29
Scraping for: 2021-01-30
Scraping for: 2021-01-31
Scraping for: 2021-02-01
Scraping for: 2021-02-02
Scraping for: 2021-02-03
Scraping for: 2021-02-04
Scraping for: 2021-02-05
Scraping for: 2021-02-06
Scraping for: 2021-02-07
Scraping for: 2021-02-08
Scraping for: 2021-02-09


,Team_x,Points_x,Team_y,Points_y,Date,Home,Winner
0,Atlanta,114,Brooklyn,96,2021-01-01,Brooklyn,Atlanta
1,Brooklyn,96,Atlanta,114,2021-01-01,Brooklyn,Atlanta
2,Memphis,108,Charlotte,93,2021-01-01,Charlotte,Memphis
3,Charlotte,93,Memphis,108,2021-01-01,Charlotte,Memphis
4,Miami,83,Dallas,93,2021-01-01,Dallas,Dallas
...,...,...,...,...,...,...,...
2187,Phoenix,102,LA Clippers,116,2021-06-28,Phoenix,LA Clippers
2188,Milwaukee,88,Atlanta,110,2021-06-29,Atlanta,Atlanta
2189,Atlanta,110,Milwaukee,88,2021-06-29,Atlanta,Atlanta
2190,Phoenix,130,LA Clippers,103,2021-06-30,LA Clippers,Phoenix


In [60]:
def fix_okc(name):
    if name == 'Okla City':
        return 'Oklahoma City'
    return name

In [61]:
def scrape_data_averages_for():
   
    url = 'https://www.teamrankings.com/nba/stat/points-per-game?date='
    columns = ['Team', 'Season Ave', '3 Game Ave', 'Last Game Ave', 'Home Ave', 'Away Ave', 'Last Season Ave', 'Date']
    float_vals = ['Season Ave', '3 Game Ave', 'Last Game Ave', 'Home Ave', 'Away Ave', 'Last Season Ave']
    temp = []
    for date_str in get_date_array():
        if date_str in finals.Date.tolist():
            print('Scraping for: %s' % date_str)
            html = urllib.urlopen(url + date_str)
            soup = BeautifulSoup(html.read())
            html.close()
            rows = soup.find_all('tr')
            for row in rows[1:]:
                line=[]
                row = row.get_text()
                line = row.split('\n')
                for c in range(len(line)):
                    if line[c] == '--':
                        line[c] = '-99'
                line = line[2:len(line)-1]
                line.append(date_str)
                temp.append(line)
            sleep(randint(1,3))
    ret = pd.DataFrame(temp, columns = columns)
    ret[float_vals] = ret[float_vals].astype(float)
    ret['Team'] = ret['Team'].map(fix_okc)
    ret= ret.replace({-99.0: 'NaN'})
    return ret.drop("Last Season Ave", axis=1)

In [62]:
def scrape_data_averages_against():
   
    url = 'https://www.teamrankings.com/nba/stat/opponent-points-per-game?date='
    columns = ['Team', 'Def Season Ave', 'Def 3 Game Ave', 'Def Last Game Ave', 'Def Home Ave', 'Def Away Ave', 'Def Last Season Ave', 'Date']
    float_vals = ['Def Season Ave', 'Def 3 Game Ave', 'Def Last Game Ave', 'Def Home Ave', 'Def Away Ave', 'Def Last Season Ave']
    temp = []
    for date_str in get_date_array():
        if date_str in finals.Date.tolist():
            print('Scraping for: %s' % date_str)
            html = urllib.urlopen(url + date_str)
            soup = BeautifulSoup(html.read())
            html.close()
            rows = soup.find_all('tr')
            for row in rows[1:]:
                line=[]
                row = row.get_text()
                line = row.split('\n')
                for c in range(len(line)):
                    if line[c] == '--':
                        line[c] = '-99'
                line = line[2:len(line)-1]
                line.append(date_str)
                temp.append(line)
            sleep(randint(1,3))
    ret = pd.DataFrame(temp, columns = columns)
    ret[float_vals] = ret[float_vals].astype(float)
    ret['Team'] = ret['Team'].map(fix_okc)
    ret= ret.replace({-99.0: 'NaN'})
    return ret.drop("Def Last Season Ave", axis=1)

In [63]:
averages_for = scrape_data_averages_for()
averages_against = scrape_data_averages_against()

Scraping for: 2021-01-01
Scraping for: 2021-01-02
Scraping for: 2021-01-03
Scraping for: 2021-01-04
Scraping for: 2021-01-05
Scraping for: 2021-01-06
Scraping for: 2021-01-07
Scraping for: 2021-01-08
Scraping for: 2021-01-09
Scraping for: 2021-01-10
Scraping for: 2021-01-11
Scraping for: 2021-01-12
Scraping for: 2021-01-13
Scraping for: 2021-01-14
Scraping for: 2021-01-15
Scraping for: 2021-01-16
Scraping for: 2021-01-17
Scraping for: 2021-01-18
Scraping for: 2021-01-19
Scraping for: 2021-01-20
Scraping for: 2021-01-21
Scraping for: 2021-01-22
Scraping for: 2021-01-23
Scraping for: 2021-01-24
Scraping for: 2021-01-25
Scraping for: 2021-01-26
Scraping for: 2021-01-27
Scraping for: 2021-01-28
Scraping for: 2021-01-29
Scraping for: 2021-01-30
Scraping for: 2021-01-31
Scraping for: 2021-02-01
Scraping for: 2021-02-02
Scraping for: 2021-02-03
Scraping for: 2021-02-04
Scraping for: 2021-02-05
Scraping for: 2021-02-06
Scraping for: 2021-02-07
Scraping for: 2021-02-08
Scraping for: 2021-02-09


Scraping for: 2021-06-10
Scraping for: 2021-06-11
Scraping for: 2021-06-12
Scraping for: 2021-06-13
Scraping for: 2021-06-14
Scraping for: 2021-06-15
Scraping for: 2021-06-16
Scraping for: 2021-06-17
Scraping for: 2021-06-18
Scraping for: 2021-06-19
Scraping for: 2021-06-20
Scraping for: 2021-06-22
Scraping for: 2021-06-23
Scraping for: 2021-06-24
Scraping for: 2021-06-25
Scraping for: 2021-06-26
Scraping for: 2021-06-27
Scraping for: 2021-06-28
Scraping for: 2021-06-29
Scraping for: 2021-06-30


In [64]:
averages_for.head()

,Team,Season Ave,3 Game Ave,Last Game Ave,Home Ave,Away Ave,Date
0,Atlanta,128.8,130.3,141.0,128.0,129.0,2021-01-01
1,Milwaukee,124.2,120.7,108.0,138.0,120.8,2021-01-01
2,Brooklyn,121.6,120.0,145.0,127.0,113.5,2021-01-01
3,LA Lakers,120.4,118.3,121.0,120.3,121.0,2021-01-01
4,Houston,119.7,119.7,122.0,122.0,118.5,2021-01-01


In [65]:
averages_against.head()

,Team,Def Season Ave,Def 3 Game Ave,Def Last Game Ave,Def Home Ave,Def Away Ave,Date
0,Phoenix,97.8,93.7,95.0,94.0,100.3,2021-01-01
1,New Orleans,99.2,95.3,80.0,95.0,100.3,2021-01-01
2,Philadelphia,99.8,101.0,92.0,100.0,99.7,2021-01-01
3,Toronto,103.8,100.7,83.0,98.0,109.5,2021-01-01
4,New York,105.2,98.7,100.0,109.5,102.3,2021-01-01


In [66]:
def get_all_data():
    test = pd.merge(finals, averages_for, left_on=['Team_x', 'Date'], right_on=['Team', 'Date'], how='left').dropna()
    test = test.drop('Team', axis=1).dropna()
    test_2 = pd.merge(test, averages_for, left_on=['Team_y', 'Date'], right_on=['Team', 'Date'], how='left').dropna()
    test_2 = test_2.drop('Team', axis=1).dropna()
    test_3 = pd.merge(test_2, averages_against, left_on=['Team_x', 'Date'], right_on=['Team', 'Date'], how='left').dropna()
    test_3 = test_3.drop('Team', axis=1).dropna()
    test_4 = pd.merge(test_3, averages_against, left_on=['Team_y', 'Date'], right_on=['Team', 'Date'], how='left').dropna()
    return test_4.drop('Team', axis=1).dropna()

In [67]:
df = get_all_data()
df = df.dropna(how='any')
df

,Team_x,Points_x,Team_y,Points_y,Date,Home,Winner,Season Ave_x,3 Game Ave_x,Last Game Ave_x,...,Def Season Ave_x,Def 3 Game Ave_x,Def Last Game Ave_x,Def Home Ave_x,Def Away Ave_x,Def Season Ave_y,Def 3 Game Ave_y,Def Last Game Ave_y,Def Home Ave_y,Def Away Ave_y
0,Atlanta,114,Brooklyn,96,2021-01-01,Brooklyn,Atlanta,128.8,130.3,141.0,...,120.3,125.7,145.0,120.0,120.3,111.4,121.0,141.0,118.7,100.5
1,Brooklyn,96,Atlanta,114,2021-01-01,Brooklyn,Atlanta,121.6,120.0,145.0,...,111.4,121.0,141.0,118.7,100.5,120.3,125.7,145.0,120.0,120.3
2,Memphis,108,Charlotte,93,2021-01-01,Charlotte,Memphis,113.5,111.7,107.0,...,122.5,119.7,126.0,126.5,118.5,108.3,104.0,99.0,106.5,110.0
3,Charlotte,93,Memphis,108,2021-01-01,Charlotte,Memphis,111.3,110.3,118.0,...,108.3,104.0,99.0,106.5,110.0,122.5,119.7,126.0,126.5,118.5
4,Miami,83,Dallas,93,2021-01-01,Dallas,Dallas,108.5,109.0,119.0,...,115.8,116.7,108.0,116.7,113.0,108.8,109.7,118.0,118.0,105.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,Phoenix,102,LA Clippers,116,2021-06-28,Phoenix,LA Clippers,114.1,93.3,84.0,...,108.0,96.3,80.0,106.4,109.7,107.6,93.3,84.0,107.6,107.5
2188,Milwaukee,88,Atlanta,110,2021-06-29,Atlanta,Atlanta,118.5,117.0,113.0,...,112.3,103.0,102.0,110.5,114.1,110.7,117.0,113.0,108.6,112.6
2189,Atlanta,110,Milwaukee,88,2021-06-29,Atlanta,Atlanta,112.3,103.0,102.0,...,110.7,117.0,113.0,108.6,112.6,112.3,103.0,102.0,110.5,114.1
2190,Phoenix,130,LA Clippers,103,2021-06-30,LA Clippers,Phoenix,113.9,92.7,102.0,...,108.1,100.7,116.0,106.6,109.7,107.5,92.7,102.0,107.6,107.4


In [68]:
def is_same(col):
    team = col.split(', ')[0]
    target = col.split(', ')[1]
    if team==target:
        return 1
    else: return 0

In [69]:
import numpy as np
def messy_wrangling():
    ret=pd.DataFrame()
    ret['Home?'] = df['Team_x'] + ', ' + df['Home']
    ret['Win?'] = df['Team_x'] + ', ' + df['Winner']
    ret['Home?'] = ret['Home?'].map(is_same)
    ret['Win?'] = ret['Win?'].map(is_same)
    
    ret['Offensive Adv Season'] = df['Season Ave_x'] - df['Def Season Ave_y']
    ret['Offensive Adv 3G'] = df['3 Game Ave_x'] - df['Def 3 Game Ave_y']
    ret['Offensive Adv 1G'] = df['Last Game Ave_x'] - df['Def Last Game Ave_y']

    
    ret['Defensive Adv Season'] = df['Def Season Ave_x'] - df['Season Ave_y']
    ret['Defensive Adv 3G'] = df['Def 3 Game Ave_x'] - df['3 Game Ave_y']
    ret['Defensive Adv 1G'] = df['Def Last Game Ave_x'] - df['Last Game Ave_y']
    
    ret['Offensive Home Adv'] = df['Home Ave_x'] - df['Def Away Ave_y']
    ret['Offensive Away Adv'] = df['Away Ave_x'] - df['Def Home Ave_y']
    ret['Defensive Home Adv'] = df['Def Home Ave_x'] - df['Away Ave_y']
    ret['Defensive Away Adv'] = df['Def Away Ave_x'] - df['Home Ave_y']
    
    ret['Offensive H/A Adv'] = np.where(ret['Home?']==1, ret['Offensive Home Adv'], ret['Offensive Away Adv'])
    ret['Defensive H/A Adv'] = np.where(ret['Home?']==1, ret['Defensive Home Adv'], ret['Defensive Away Adv'])

    return ret.drop(['Offensive Home Adv', 'Offensive Away Adv', 'Defensive Home Adv', 'Defensive Away Adv'], axis=1)

In [70]:
final=messy_wrangling()
final

TypeError: unsupported operand type(s) for -: 'str' and 'float'

In [ ]:
final.to_csv(CSV_TITLE)